In [1]:
import os
import numpy as np

import torch
torch.manual_seed(29)
from torch import nn
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import torch.nn.parallel
cudnn.benchmark = True
from torch.utils.data import DataLoader

from torchstat import stat
from utils import  models
from utils.data import dataset_1
from utils.trainer_utils import parfilter

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from skimage.measure import compare_psnr, compare_ssim

from matplotlib import pyplot as plt

from glob import glob
from tqdm import tqdm

from PIL.PngImagePlugin import PngImageFile, PngInfo

In [2]:
for i in list(np.arange(0, 256, 32)):
    for j in list(np.arange(0, 192, 32)):
        print([i-4, j-4, (i+32)+4, (j+32)+4])

[-4, -4, 36, 36]
[-4, 28, 36, 68]
[-4, 60, 36, 100]
[-4, 92, 36, 132]
[-4, 124, 36, 164]
[-4, 156, 36, 196]
[28, -4, 68, 36]
[28, 28, 68, 68]
[28, 60, 68, 100]
[28, 92, 68, 132]
[28, 124, 68, 164]
[28, 156, 68, 196]
[60, -4, 100, 36]
[60, 28, 100, 68]
[60, 60, 100, 100]
[60, 92, 100, 132]
[60, 124, 100, 164]
[60, 156, 100, 196]
[92, -4, 132, 36]
[92, 28, 132, 68]
[92, 60, 132, 100]
[92, 92, 132, 132]
[92, 124, 132, 164]
[92, 156, 132, 196]
[124, -4, 164, 36]
[124, 28, 164, 68]
[124, 60, 164, 100]
[124, 92, 164, 132]
[124, 124, 164, 164]
[124, 156, 164, 196]
[156, -4, 196, 36]
[156, 28, 196, 68]
[156, 60, 196, 100]
[156, 92, 196, 132]
[156, 124, 196, 164]
[156, 156, 196, 196]
[188, -4, 228, 36]
[188, 28, 228, 68]
[188, 60, 228, 100]
[188, 92, 228, 132]
[188, 124, 228, 164]
[188, 156, 228, 196]
[220, -4, 260, 36]
[220, 28, 260, 68]
[220, 60, 260, 100]
[220, 92, 260, 132]
[220, 124, 260, 164]
[220, 156, 260, 196]


In [2]:
def image_quality_comp(I, Iout, Enout):
    
    for x in ['test.png', 'test_35.jpg', 'test.flif', 'test1.png', 'test1.flif', 'test_comp.png']: 
        if os.path.exists(x):os.remove(x)
    
    I.save('test.png')
    Iout.save('test_comp.png')
    I.save('test_10.jpg', quality = 10)
    Enout.save('test1.png')
    
    os.system("flif -e test.png test.flif")
    os.system("flif -e test1.png test1.flif")
    
    print('Original Image :: ')
    print('PNG     :: ' + str(8*os.path.getsize('test.png')/(I.size[0] * I.size[1])))
    print('JPG-10% :: ' + str(8*os.path.getsize('test_10.jpg')/(I.size[0] * I.size[1])))
    print('FLIF    :: ' + str(8*os.path.getsize('test.flif')/(I.size[0] * I.size[1])))
    print('Encoded Image :: ')
    print('PNG     :: ' + str(8*os.path.getsize('test1.png')/(I.size[0] * I.size[1])))
    print('FLIF    :: ' + str(8*os.path.getsize('test1.flif')/(I.size[0] * I.size[1])))
    
def image_quality_comp2(I, Iout):
    
    for x in ['test.png', 'test_35.jpg', 'test.flif', 'test1.png', 'test1.flif', 'test_comp.png']: 
        if os.path.exists(x):os.remove(x)
    
    I.save('test.png')
    Iout.save('test_comp.png')
    I.save('test_10.jpg', quality = 10)
    
    os.system("flif -e test.png test.flif")
    
    print('Original Image :: ')
    print('PNG     :: ' + str(8*os.path.getsize('test.png')/(I.size[0] * I.size[1])))
    print('JPG-10% :: ' + str(8*os.path.getsize('test_10.jpg')/(I.size[0] * I.size[1])))
    print('FLIF    :: ' + str(8*os.path.getsize('test.flif')/(I.size[0] * I.size[1])))

In [3]:
def compress_and_decompress(img, model, src_fldr, dst_fldr):
    I_org = Image.open(img).convert('RGB')
    
    s_ = 512
    d_ = 128
    
    w, h = I_org.size
    I = np.uint8(I_org).copy()
    I = np.pad(I, ((0, int(s_*np.ceil(I.shape[0]/s_) - I.shape[0])), 
                   (0, int(s_*np.ceil(I.shape[1]/s_) - I.shape[1])), 
                   (0, 0)), mode = "reflect")
    I = Image.fromarray(I)
    
    
    I1 = np.float32(I)#[:256, :256, :]
    I1 = np.transpose(I1, [2, 0, 1])
    Iout = np.zeros_like(I1)
    for i in range(0, I1.shape[1], s_):
        for j in range(0, I1.shape[2], s_):
            It = torch.from_numpy(np.expand_dims(I1[:, i:i+s_, j:j+s_], 0))/255.0
            X = model(It.cuda())
            X = X.data.squeeze().cpu().numpy()
            Iout[:, i:i+s_, j:j+s_] = np.clip(X, 0, 1)
            
    Iout = np.uint8(255 * Iout.transpose([1, 2, 0]))
    Iout = Image.fromarray(Iout).crop((0, 0, w, h))
    
    Iout.save(img.replace(src_fldr, dst_fldr))
    return Iout

def compress_and_decompress2(img, model):
    I_org = Image.open(img).convert('RGB')
    
    s_ = 512
    d_ = 128
    
    w, h = I_org.size
    I = np.uint8(I_org).copy()
    I = np.pad(I, ((0, int(s_*np.ceil(I.shape[0]/s_) - I.shape[0])), 
                   (0, int(s_*np.ceil(I.shape[1]/s_) - I.shape[1])), 
                   (0, 0)), mode = "reflect")
    I = Image.fromarray(I)
    
    
    I1 = np.float32(I)#[:256, :256, :]
    I1 = np.transpose(I1, [2, 0, 1])
    Iout = np.zeros_like(I1)
    for i in range(0, I1.shape[1], s_):
        for j in range(0, I1.shape[2], s_):
            It = torch.from_numpy(np.expand_dims(I1[:, i:i+s_, j:j+s_], 0))/255.0
            X = model(It.cuda())
            X = X.data.squeeze().cpu().numpy()
            Iout[:, i:i+s_, j:j+s_] = np.clip(X, 0, 1)
            
    Iout = np.uint8(255 * Iout.transpose([1, 2, 0]))
    Iout = Image.fromarray(Iout).crop((0, 0, w, h))
    
    
    return Iout

def compress2(img, model):
    I_org = Image.open(img).convert('RGB')
    
    s_ = 512
    d_ = 128
    
    w, h = I_org.size
    I = np.uint8(I_org).copy()
    I = np.pad(I, ((0, int(s_*np.ceil(I.shape[0]/s_) - I.shape[0])), 
                   (0, int(s_*np.ceil(I.shape[1]/s_) - I.shape[1])), 
                   (0, 0)), mode = "reflect")
    I = Image.fromarray(I)
    
    
    I1 = np.float32(I)#[:256, :256, :]
    I1 = np.transpose(I1, [2, 0, 1])
    Eout = []
    for i in range(0, I1.shape[1], s_):
        for j in range(0, I1.shape[2], s_):
            It = torch.from_numpy(np.expand_dims(I1[:, i:i+s_, j:j+s_], 0))/255.0
            X = model.encode(It.cuda())
            X = X.data.squeeze().cpu().numpy()
            Eout.append(X)
            
    return np.concatenate(Eout)

In [4]:
def perform_compression(I_org, model):
    
    w, h = I_org.size
    I = np.uint8(I_org).copy()
    I = np.pad(I, ((0, int(256*np.ceil(I.shape[0]/256) - I.shape[0])), 
                   (0, int(256*np.ceil(I.shape[1]/256) - I.shape[1])), 
                   (0, 0)), mode = "reflect")
    I = Image.fromarray(I)
    
    
    s_ = 512
    d_ = 512//16
    
    c_ = s_//d_
    
    I1 = np.float32(I)#[:256, :256, :]
    I1 = np.transpose(I1, [2, 0, 1])
    Iout = np.zeros_like(I1)
    Enout = np.zeros((3, I1.shape[1]//c_, I1.shape[2]//c_))
    for i in range(0, I1.shape[1], s_):
        for j in range(0, I1.shape[2], s_):
            It = torch.from_numpy(np.expand_dims(I1[:, i:i+s_, j:j+s_], 0))/255.0
            Xe = model.encode(It.cuda())
            X = model.decode(Xe)
            X = X.data.squeeze().cpu().numpy()
            Iout[:, i:i+s_, j:j+s_] = np.clip(X, 0, 1)
            Xe = (Xe + 1)/2
            Enout[:, i//c_:(i+s_)//c_, j//c_:(j+s_)//c_] = Xe.data.squeeze().cpu().numpy()
    
    
    Iout = np.uint8(255 * Iout.transpose([1, 2, 0]))
    Enout = np.uint8(255 * Enout.transpose([1, 2, 0]))
    
    Iout = Image.fromarray(Iout).crop((0, 0, w, h))
    Enout = Image.fromarray(Enout)
    
    psnr = compare_psnr(np.uint8(I_org).copy(), np.uint8(Iout).copy())
#     ssim = compare_ssim(np.uint8(I_org).copy(), np.uint8(Iout).copy())
    
    print('PSNR-PROP :: ' + "{0:0.02f}".format(psnr))
#     print('SSIM-PROP :: ' + "{0:0.02f}".format(ssim), multichannel=True)
    
    image_quality_comp(I_org, Iout, Enout)
    psnr = compare_psnr(np.uint8(I_org).copy(), np.uint8(Image.open('test_10.jpg')).copy())
    print('PSNR-JPG10 :: ' + "{0:0.02f}".format(psnr))
    
    
    return Iout, Enout

In [5]:
def perform_compression2(I_org, model):
    
    c_ = 8
    
    w, h = I_org.size
    I = np.uint8(I_org).copy()
    I = np.pad(I, ((0, int(c_*np.ceil(I.shape[0]/c_) - I.shape[0])), 
                   (0, int(c_*np.ceil(I.shape[1]/c_) - I.shape[1])), 
                   (0, 0)), mode = "reflect")
    I = Image.fromarray(I)
    
    
    I1 = np.float32(I)#[:256, :256, :]
    I1 = np.transpose(I1, [2, 0, 1])/255.0
    Xe = model.encode(torch.from_numpy(np.expand_dims(I1, 0)).cuda())
    X = model.decode(Xe)
    X = X.data.squeeze().cpu().numpy()
    Iout = np.clip(X, 0, 1)
    Xe = (Xe + 1)/2
    Enout = Xe.data.squeeze().cpu().numpy()
    
    Iout = np.uint8(255 * Iout.transpose([1, 2, 0]))
    Enout = np.uint8(255 * Enout.transpose([1, 2, 0]))
    
    Iout = Image.fromarray(Iout).crop((0, 0, w, h))
    Enout = Image.fromarray(Enout)
    
    psnr = compare_psnr(np.uint8(I_org).copy(), np.uint8(Iout).copy())
#     ssim = compare_ssim(np.uint8(I_org).copy(), np.uint8(Iout).copy())
    
    print('PSNR-PROP :: ' + "{0:0.02f}".format(psnr))
#     print('SSIM-PROP :: ' + "{0:0.02f}".format(ssim), multichannel=True)
    
    image_quality_comp(I_org, Iout, Enout)
    psnr = compare_psnr(np.uint8(I_org).copy(), np.uint8(Image.open('test_10.jpg')).copy())
    print('PSNR-JPG10 :: ' + "{0:0.02f}".format(psnr))
    
    
    return Iout, Enout

In [6]:
img_file = "/media/cibitaw1/DATA/super_rez/professional_valid/valid/alberto-montalesi-176097.png"

In [7]:
I = Image.open(img_file).convert("RGB")

In [8]:
model = models.QuantACTShuffleV7()
check_point_file = "/media/cibitaw1/DATA/SP2020/compressACT/weights/"+\
"QuantACTShuffleV7_exp01/checkpoint.pth.tar"
checkpoint = torch.load(check_point_file)
model.load_state_dict(checkpoint['state_dict'], strict = False)
model.cuda()
model.eval()
print('.')

.


In [ ]:
M = models.QuantACTShuffleV3()
check_point_file = "/media/narsi/LargeData/SP_2020/compressACT/weights/QuantACTShuffleV3_exp02/model_best.pth.tar"
checkpoint = torch.load(check_point_file)
M.load_state_dict(checkpoint['state_dict'], strict = False)

model = models.CleanImg(M)
check_point_file = "/media/narsi/LargeData/SP_2020/compressACT/weights/CleanImg_exp01/model_best.pth.tar"
checkpoint = torch.load(check_point_file)
model.load_state_dict(checkpoint['state_dict'], strict = False)

In [9]:
Iout, Enout = perform_compression(I, model)

PSNR-PROP :: 30.43
Original Image :: 
PNG     :: 12.19391966250917
JPG-10% :: 0.1823585381511372
FLIF    :: 10.78488112160675
Encoded Image :: 
PNG     :: 0.07856061995597946
FLIF    :: 0.07661465975788702
PSNR-JPG10 :: 30.35


In [ ]:
Iout = compress_and_decompress2(img_file, model)
image_quality_comp(I, Iout)
psnr = compare_psnr(np.uint8(I).copy(), np.uint8(Iout).copy())
print('PSNR-PROP :: ' + "{0:0.02f}".format(psnr))
psnr = compare_psnr(np.uint8(I).copy(), np.uint8(Image.open('test_10.jpg')).copy())
print('PSNR-JPG35 :: ' + "{0:0.02f}".format(psnr))

In [ ]:
eout = compress2(img_file, model)
eout = np.uint8(eout == 1).reshape(-1)

In [ ]:
len(eout)/(I.size[0] * I.size[1])

In [ ]:
dist = plt.hist(eout, bins = 2)
print(dist[0]/np.sum(dist[0]))

In [ ]:
from range_coder import RangeEncoder, RangeDecoder, prob_to_cum_freq

data = list(eout.tolist())
prob = list(dist[0]/np.sum(dist[0]))

# convert probabilities to cumulative integer frequency table
cumFreq = prob_to_cum_freq(prob, resolution=128)

filepath = 'test.dat'

# encode data
encoder = RangeEncoder(filepath)
encoder.encode(data, cumFreq)
encoder.close()

# decode data
decoder = RangeDecoder(filepath)
dataRec = decoder.decode(len(data), cumFreq)
decoder.close()


In [ ]:
(os.path.getsize('test.dat')*8)/(I.size[0] * I.size[1])

In [ ]:
model = models.QuantACTShuffleV5()
model.cuda()
%timeit model.encode(torch.randn(1, 3, 256, 256).cuda())
%timeit model.decode(torch.randn(1, 3, 64, 64).cuda())

In [ ]:
model = models.QuantACTShuffleV5()
%timeit model.encode(torch.randn(1, 3, 256, 256))
%timeit model.decode(torch.randn(1, 3, 64, 64))

In [ ]:
src_fldr = "/media/cibitaw1/DATA/super_rez/professional_valid/valid"
dst_fldr = "/media/cibitaw1/DATA/super_rez/professional_valid/model_test"

imgs = glob(src_fldr + os.sep + "*.png")

In [ ]:
model.cuda()
print('.')

In [ ]:
for img in tqdm(imgs):
    compress_and_decompress(img, model, src_fldr, dst_fldr)

In [ ]:
compressed_imgs = [img.replace(src_fldr, dst_fldr) for img in imgs]

In [ ]:
from utils.competition_eval import evaluate2

In [ ]:
results = evaluate2(compressed_imgs, imgs)

In [ ]:
results

In [ ]:
results

In [ ]:
(29.310540294207478+28.83994787307876)/2

In [ ]:
(0.9736846666083886+0.9705319931942877)/2